In [1]:
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import wordninja
from wordninja import LanguageModel
import re
import numpy as np
import gzip
import shutil

# 資料讀取
df = pd.read_excel('../data/combined_excel.xlsx',index_col=0).astype(str)
df = df.rename(columns={'45A':'string_X_train','SPEC':'Y_label'})
df = df[['string_X_train','Y_label','EXPNO']].dropna(axis=0)
print(df.shape)
df.head(10)

(20737, 3)


,string_X_train,Y_label,EXPNO
0,SHIPMENT OFPVC SUSPENSION RESIN S-65D QTY 18 M...,nan,nan
1,"17 MT PVC COPOLYMER RESIN, C-15 AT USD 1470 ...",nan,nan
2,"+TERMS OF SALE: CIF NHAVA SHEVA PORT, INDIA++7...",nan,nan
3,"MASS PVC RESIN, B-57QUANTITY 175 MT AT 1300 US...",MASS PVC RESIN B-57,11
4,PHTHALIC ANHYDRIDE (PA)QUANTITY = 306 MT UNIT ...,(鄰苯二甲酐)\nPHTHALIC ANHYDRIDE,27
5,+ COMMODITY: LLDPE TAISOX 3470+ QUANTITY: 32.0...,LINEAR LOW DENSITY\nPOLYETHYLENE RESIN\nTAISOX...,18
6,"+TERM OF SALE: CIF MUNDRA SEAPORT, INDIA++70 M...",nan,nan
7,COMMODITY ...,POLYESTER PARTIALY\nORIENTED YARN,25
8,COMMODITY ...,POLYESTER PARTIALY\nORIENTED YARN,25
9,COMMODITY ...,POLYESTER FILAMENT YARN,25


In [2]:
# 可以觀察到有些LABEL是NAN所以取得寶典 來幫忙標註
b1 = pd.read_excel('../data\台塑企業_ 產品寶典20210303.xlsx').iloc[:,:5]
b2 = pd.read_excel('../data\寶典.v3.台塑網.20210901.xlsx').iloc[:,:5]
b2.columns = b1.columns
b3 = b1.append(b2)
寶典 = b3['品名'].values.tolist() + [ i for i in df['Y_label'].values.tolist() if ("\n" not in i)]
寶典 = set(寶典)
list(寶典)[:10]

['PVC RESIN S 65D',
 'HHPA (HEXA HYDRO PHTHALIC ANHYDRIDE)',
 'COPOLYMER PVC C-8',
 'RAW MATERIAL FOR ZIPPER INDUSTRIES',
 'CAC 03',
 'PP SHEETING',
 'FORMOTEX',
 'ETHYLENE PROPYLENE COPOLYMER 3005',
 'RAYON TOW',
 'CARBON FIBER TC-35']

In [3]:
def Collection_method(df,產品集合):
    labels = {}
    for i in tqdm(df.index):
        products = []
        for p in 產品集合:
            if p in df.loc[i,'string_X_train']:
                products.append(p) # 加入候選清單
        try:
            labels[i] = max(products,key=len) # 選清單中最長的產品
        except:
            labels[i] = np.nan # 真的找不到就算了
    predict = pd.DataFrame(index=labels.keys(),columns=['Baodian_predict'])
    predict['Baodian_predict'] = labels.values()
    return predict

# 執行寶典搜索
predict = Collection_method(df,寶典)
# 將寶典標註結果加入df
df = df.join(predict).dropna(axis=0)
df.head()

<ipython-input-3-85c5bb173add>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(df.index):


,string_X_train,Y_label,EXPNO,Baodian_predict
0,SHIPMENT OFPVC SUSPENSION RESIN S-65D QTY 18 M...,nan,nan,PVC SUSPENSION RESIN S-65
1,"17 MT PVC COPOLYMER RESIN, C-15 AT USD 1470 ...",nan,nan,PVC COPOLYMER RESIN
2,"+TERMS OF SALE: CIF NHAVA SHEVA PORT, INDIA++7...",nan,nan,PVC RESIN
3,"MASS PVC RESIN, B-57QUANTITY 175 MT AT 1300 US...",MASS PVC RESIN B-57,11,"PVC RESIN, B-57"
4,PHTHALIC ANHYDRIDE (PA)QUANTITY = 306 MT UNIT ...,(鄰苯二甲酐)\nPHTHALIC ANHYDRIDE,27,PHTHALIC ANHYDRIDE


In [4]:
# 利用'Baodian_predict'更新Y_label
df['Y_label'].update(df['Baodian_predict']) # 本來Y_label的nan 如果寶典預測有就可以更新成功 如果寶典預測也是nan就沒辦法
df = df[['string_X_train','Y_label','EXPNO']].dropna(axis=0)
print(df.shape)
df.head()

(20008, 3)


,string_X_train,Y_label,EXPNO
0,SHIPMENT OFPVC SUSPENSION RESIN S-65D QTY 18 M...,PVC SUSPENSION RESIN S-65,nan
1,"17 MT PVC COPOLYMER RESIN, C-15 AT USD 1470 ...",PVC COPOLYMER RESIN,nan
2,"+TERMS OF SALE: CIF NHAVA SHEVA PORT, INDIA++7...",PVC RESIN,nan
3,"MASS PVC RESIN, B-57QUANTITY 175 MT AT 1300 US...","PVC RESIN, B-57",11
4,PHTHALIC ANHYDRIDE (PA)QUANTITY = 306 MT UNIT ...,PHTHALIC ANHYDRIDE,27


In [5]:
df = df[['string_X_train','Y_label','EXPNO']].dropna(axis=0)
df = df[df['EXPNO']!='nan']
print(df.shape)
df.head(10)

(13614, 3)


,string_X_train,Y_label,EXPNO
3,"MASS PVC RESIN, B-57QUANTITY 175 MT AT 1300 US...","PVC RESIN, B-57",11
4,PHTHALIC ANHYDRIDE (PA)QUANTITY = 306 MT UNIT ...,PHTHALIC ANHYDRIDE,27
5,+ COMMODITY: LLDPE TAISOX 3470+ QUANTITY: 32.0...,LLDPE TAISOX,18
7,COMMODITY ...,POLYESTER PARTIALY ORIENTED YARN,25
8,COMMODITY ...,POLYESTER PARTIALY ORIENTED YARN,25
9,COMMODITY ...,POLYESTER PARTIALY ORIENTED YARN,25
10,COMMODITY ...,POLYESTER PARTIALY ORIENTED YARN,25
11,COMMODITY ...,POLYESTER PARTIALY ORIENTED YARN,25
12,+ITEM1: HDPE TAISOX 8010 200MT USD1100/MTCONTR...,HDPE TAISOX 8010,18
13,+ITEM1: HDPE TAISOX 8010 200MT USD1100/MTCONTR...,HDPE TAISOX 8010,18


In [6]:
# 預處理
def preprocess(x):
    x = re.sub('[\u4e00-\u9fa5]', '', x) # 去除中文
    x = re.sub(r'[^\w\s]','',x) # 去除標點符號
    x = x.replace('\n', '').replace('\r', '').replace('\t', '')
    x = str.strip(x) # 移除左右空白
    return x

def preprocess_x(x):
    for p in 寶典:
        if p in x:
            if  x[x.find(p)-1] != ' ': # 如果跟前面字串黏再一起
                ys = x.find(p) # 找到起始點
                ye = ys + len(p) #找到結束點
                x = x[:ys] + ' ' + x[ys:ye] + x[ye:] #前文+空白+產品+後文
            
            elif x.find(p)+len(p) < len(x)-1: # 如果品名結尾位置小於字串末端
                if x[x.find(p)+len(p)+1] != ' ': # 如果品名結尾位置跟後面字串黏再一起
                    ys = x.find(p) # 找到起始點
                    ye = ys + len(p) #找到結束點
                    x = x[:ys] + x[ys:ye] + ' ' + x[ye:] #前文+產品+空白+後文
            else:
                pass
    x = preprocess(x) # 然後再接預處理
    return x
df['string_X_train'] = df['string_X_train'].apply(preprocess_x)
df['Y_label'] = df['Y_label'].apply(preprocess)

In [7]:
# 起始結束位置標註
def str2index(context,string):
    ys = context.find(string)
    ye = ys + len(string)
    return ys,ye

ys_lst,ye_lst = [],[]
for i in range(len(df)):
    ys,ye = str2index(df['string_X_train'].values[i],df['Y_label'].values[i])
    ys_lst.append(ys)
    ye_lst.append(ye)
df['string_Y_1'] = ys_lst
df['string_Y_2'] = ye_lst
print(1,df.shape)

df = df[df['Y_label']!=''] # 去掉空值
print(2,df.shape)

df = df.loc[df['string_Y_1']!=-1,:] # 去掉找不到答案的
print(3,df.shape)

df = df.dropna(subset=['string_X_train','Y_label'],axis=0) # 去掉NAN
print(4,df.shape)

df.head(10)

1 (13614, 5)
2 (13614, 5)
3 (9895, 5)
4 (9895, 5)


,string_X_train,Y_label,EXPNO,string_Y_1,string_Y_2
3,MASS PVC RESIN B57 QUANTITY 175 MT AT 1300 USD...,PVC RESIN B57,11,5,18
4,PHTHALIC ANHYDRIDE PAQUANTITY 306 MT UNIT P...,PHTHALIC ANHYDRIDE,27,0,18
16,COMMODITY TAISOX 7470MQUANTITY100MTUNIT PRICEU...,TAISOX,18,10,16
18,COUNTRY OF ORIGIN TAIWANPRICE TERMS CIF BU S...,SAN,XU,45,48
19,COUNTRY OF ORIGIN TAIWANPRICE TERMS CIF BU S...,SAN,XU,45,48
22,300000 BBLS 10PCT OF GASOIL 10 PPMUNIT PRICE ...,GASOIL,61,21,27
23,300000 BBLS 10PCT OF GASOIL 10 PPMUNIT PRICE ...,GASOIL,61,21,27
24,300000 BBLS 10PCT OF GASOIL 10 PPMUNIT PRICE ...,GASOIL,61,21,27
30,300000 BBLS 10PCT OF GASOIL 10 PPMUNIT PRICE ...,GASOIL,61,21,27
31,PRODUCT TETRAHYDROFURAN 998 PCT MINQUANTITY...,TETRAHYDROFURAN,2P,10,25


In [8]:
# 最後檢查 Y1 Y2 位置 和 Y_LABEL要對應上
for i in tqdm(df.index):
    a = df.loc[i,'string_X_train'][df.loc[i,'string_Y_1']:df.loc[i,'string_Y_2']]
    b = df.loc[i,'Y_label']
    assert a == b 

<ipython-input-8-ae9b0fbb10a3>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(df.index):


In [9]:
from termcolor import colored

# 實際感受一下
def str2index(context,string):
    if type(string) != str:
        print(string)
    ys = context.find(string)
    ye = ys + len(string)
    return ys,ye

def color_output(text_input,text_output):
    ys,ye = str2index(text_input,text_output)
    left = text_input[:ys]
    mid = text_output
    right = text_input[ye:]
    print(left,colored(mid,'red'),right)

for j in range(10):
    i = np.random.randint(low=0,high=len(df))
    x = df['string_X_train'].values[i]
    p = df['Y_label'].values[i]
    print(j)
    color_output(x,p)
    print(colored(p,'green'))

0
PA CKING MA TERIAL PHARMACEUTICAL GRADE FOR PHARMACEUTICAL USE1 QUANTITY 3000 KGS   NAN YA BRAND    RIGID PVC FILM      CL EAR TRANSPARENT   250 MICRON THICK   REEL WIDTH 245 MM   AT THE RATE USD 160 PER KG2 QUANTITY 1000 KGS   NAN YA BRAND   RIGID PVC FILM   CLEAR TRANSPARENT   250 MICRON THICK   REEL WIDTH 190 MM   AT THE RATE USD 160 PER KG3 QUANTITY 1000 KGS   NAN YA BRAND   RIGID PVC FILM   CLEAR TRANSPARENT   350 MICRON THICK   REEL WIDTH 192 MM   AT THE RATE USD 160 PER KGFOB ANY TAIWAN SEAPORT ALL OTHER DETAILS AND SPECIFICATIONS AS PER INDENT NO 2102721 DATED 15012021 OF MS TRANSCONTINENTAL AGENCIES KARACHI PAKISTAN
RIGID PVC FILM
1
LLD  PE  3224     QTY 84 MT AT USD1290MTFOB ANY TAIWANESE PORT
PE
2
 nan 
nan
3
PP  SYNTHETIC PA  PER  BCD 04E 02MM X 1100MM X 3000M 31120KGS PP SYNTHETIC PAPER   BCD 04E 018MM X 1100MM X 3000M 1400KGSUNIT PRICEUSD233KGSHIPPING MA RK LKI05821TW                         XINGANGCH INAGOODS ORIGINTAIWAN PACKINGSTANDARD EXPORT PACKINGPRICE TERMCIF XIN

In [10]:
# 保存
print(df.shape)
df.to_csv('../data/preprocess_for_SQUAD_產品.csv')

(9895, 5)
